https://boosters.pro/championship/predskazanie/overview

Hеобходимо для каждой пары команд вывести вероятности трех исходов: победа домашней команды, победа гостевой команды, ничья. 


In [ ]:
import pandas as pd
import numpy as np
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_predict, GridSearchCV 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.pipeline import Pipeline
import copy
import xgboost as xgb


# Ложное срабатывание, напоминает, что можно работать с копией а не с оригиналом.
# Его можно отключить. 
pd.options.mode.chained_assignment = None 

In [ ]:
from google.colab import files

In [ ]:
files.upload()  # train

In [ ]:
files.upload()    # test

In [ ]:
train = pd.read_csv("train.csv")                # (16456, 18)        ~2.7Mb
test = pd.read_csv("test.csv")                  # (7211, 15)         ~1Mb
#ans = pd.read_csv("sample_submission.csv")     # (7211, 4) не используется, только посмотреть и понять #Потерял этот файл.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

**Понимание признаков.**

`Красные карточки` могут влиять на `target`, но их всего лишь 3%; `жёлтые карточки` около 20%. 
`Состав команды`, `запасные`, `кто вышел на замену` - есть данные от 10 до 30%. Можно было бы разложить на отдельных игроков и попробовать понять роль отдельных футболистов, особенно локальных 'звёзд', которые способны сделать результат. Но все игроки анонимизированы - в одном матче один персонаж назван №5, в другом он же становится №18, т.е. теряется привязка к конкретной личности, и все попытки обесмысливаются. 

Поэтому перечисленные признаки не беру.

Отдельно можно пожалеть об отсутствии интересных признаков, например о продвинутой статистики типа xG и PPDA.



В данном соревновании создать новые признаки не проблема. Но применять их сразу на все имеющиеся данные (т.е. на все сезоны) неправильно! Это средняя температура даже не по палате, а по больнице. Чтобы конкретизировать использование данных **вводится понятие  `сезон` - цикл жизни турнира и жизни команды**. Тут логика в том, что для команд из топа и с низа каждый отдельный сезон складывается цельно, что чётко видно по статистическим данным. Для команд из середины таблицы это менее справедливо, хотя и там будут закономерности, связанные именно с особенностями каждого отдельного сезона.

Признак `tour` помог понять по какой формуле (осень\весна или весна\осень) проходит чемпионат (`champ`). С этим пониманием вытащил год матча из `time` и сформировал важнейший признак `season`, который представлен в виде `2012 2013`. Tour и champ не имеют пропусков данных, что можно считать намёком на важность этих признаков.




# **Стратегия**
1. Объединение `train` & `test` не нужно. Признаки для них делаются раздельно (это знание получено опытным путём).
2. Есть признаки для домашней (`home`) и выездной (`away`) команды.
2. Создали признак `season` для `train` & `test`.
3. Кроме раздельного (для `train` & `test`) создания признаков, каждый из признаков находится в двух вариантах: 'для всех сезонов в сумме' и 'для текущего сезона'.
4. Кроме `признаков` есть ещё `параметры`, которые участвуют в создании признаков.
5. Модель считается по признакам 'для текущего сезона'.
6. Вводится понятие 'маркер' (см далее). Затем создаём признаки для `test`, в соответствии с результатом маркировки/ БОльшая часть признаков окажется 'для всего сезона', меньшая 'для текущего сезона', ~7% 'среднее по `champ` + `season`.


In [ ]:
# Создаём 'season'

# пустая колонка, инициализировали
train["season"] = '' 

for i in range(0, train.shape[0], 1):
    champ = train['champ'][i]
    date = train['time'][i]
    current_year, month = date.split('-')[0], date.split('-')[1]
    
    # Единственный чемпионат, проходящий по формуле 'весна осень' и уместившийся в один календарный год
    # Скорее всего, скандинавы какие-нибудь
    if champ == 'champ7':
        train["season"][i] = current_year  
        
    # 13 чемпионатов кроме №11 проходят по формуле 'осень весна'
    elif champ != 'champ11' and champ !=  'champ7':  
        # водораздел всегда июнь, все страны отдыхают
        if int(month) < 6:
            second_year = current_year
            first_year = int(current_year) - 1
            train["season"][i] = str(first_year) + ' ' + str(second_year) 
        else:
            first_year = current_year
            second_year = int(current_year) + 1
            train["season"][i] = str(first_year) + ' ' + str(second_year)   
    
    # Единственный чемпионат, выглядящий странно:      
    # Первый сезон 2011-12 проходит 'весна осень', остальные 'осень весна'
    # ... и это ... та-дам!... чемпионат России, перешедший с ВО на ОВ как раз в 2011
    elif champ == 'champ11':
        if int(current_year) == 2011:  
            train["season"][i] = '2011 2012'
        elif int(current_year) == 2012 and int(month) < 6:
            train["season"][i] = '2011 2012'
        elif int(current_year) == 2012 and int(month) > 6:
            train["season"][i] = '2012 2013'
        elif int(current_year) > 2012:
            if int(month) < 6:
                first_year = int(current_year) - 1
                second_year = current_year
                train["season"][i] = str(first_year) + ' ' + str(second_year) 
            else:
                first_year = current_year
                second_year = int(current_year) + 1
                train["season"][i] = str(first_year) + ' ' + str(second_year)

In [ ]:
# То же самое для test

test["season"] = '' 

for i in range(0, test.shape[0], 1):
    champ = test['champ'][i]
    date = test['time'][i]
    current_year, month = date.split('-')[0], date.split('-')[1]
    
    if champ == 'champ7':
        test["season"][i] = current_year  
        
    elif champ != 'champ11' and champ !=  'champ7':  
        if int(month) < 6:
            second_year = current_year
            first_year = int(current_year) - 1
            test["season"][i] = str(first_year) + ' ' + str(second_year) 
        else:
            first_year = current_year
            second_year = int(current_year) + 1
            test["season"][i] = str(first_year) + ' ' + str(second_year)   
    
    elif champ == 'champ11':
        if int(current_year) == 2011:  
            test["season"][i] = '2011 2012'
        elif int(current_year) == 2012 and int(month) < 6:
            test["season"][i] = '2011 2012'
        elif int(current_year) == 2012 and int(month) > 6:
            test["season"][i] = '2012 2013'
        elif int(current_year) > 2012:
            if int(month) < 6:
                first_year = int(current_year) - 1
                second_year = current_year
                test["season"][i] = str(first_year) + ' ' + str(second_year) 
            else:
                first_year = current_year
                second_year = int(current_year) + 1
                test["season"][i] = str(first_year) + ' ' + str(second_year)

# **Признаки для train**



Признак `Средний target`, т.е. среднее число очков, набираемое командой.

In [ ]:
# team_home
# one season
train["f1-1"] = train.groupby(["team_home", "season"])["target"].transform("mean") 
# all seasons
train['f1-2'] = train.team_home.map(train.groupby("team_home").target.mean()) 

# team_away
# one season
train["f2-1"] = train.groupby(["team_away", "season"])["target"].transform("mean") 
# all seasons
train['f2-2'] = train.team_away.map(train.groupby("team_away").target.mean()) 

Изредка данные 'one season' & 'all_seasons' совпадают. Не углублялся, но поверхностный взгляд показал, что для данной команды это вообще одна игра в наборе данных.

In [ ]:
#train[["f1-1","f1-2","f2-1","f2-2"]].head()

параметр (не признак) `'количество игр команды'`

In [ ]:
# за сезон

train['team_home_games_one_season'] = train.groupby(['team_home', 'season'])['target'].transform('size')
train['team_away_games_one_season'] = train.groupby(['team_away', 'season'])['target'].transform('size')
# train.groupby('team_home').size()

# за все сезоны суммарно
train['team_home_games_all_seasons'] = train.groupby(['team_home'])['target'].transform('size')
train['team_away_games_all_seasons'] = train.groupby(['team_away'])['target'].transform('size')
# Как он выглядит для всех команд сразу (другое представление)
#data.groupby(["team_home", "season"]).size()

параметр (не признак) `'Количество побед\поражений\ничьих'`

In [ ]:
# Столбец для всех исходов target
# Если в строке, например, target=0 то в этом столбце будет "сумма игр с таким исходом этой команды в этом сезоне"

# by season
train["team_home_target_one_season"] = train.groupby(['team_home','season','target'])['target'].transform('size')
train["team_away_target_one_season"] = train.groupby(['team_away','season','target'])['target'].transform('size')

# То же самое но суммарно за все сезоны для этой команды
train["team_home_target_all_seasons"] =  train.groupby(['team_home', 'target'])['target'].transform('size')
train["team_away_target_all_seasons"] = train.groupby(['team_away', 'target'])['target'].transform('size')

Признак `'Доля побед\поражений\ничьих'`

In [ ]:
# team_home
# one season
train['f3-1'] = train['team_home_target_one_season'] / train['team_home_games_one_season']   # by season
# all seasons
train['f3-2'] = train['team_home_target_all_seasons'] / train['team_home_games_all_seasons']   # all seasons

# team_away
# one season
train['f4-1'] = train['team_away_target_one_season'] / train['team_away_games_one_season']   # by season
# all seasons
train['f4-2'] = train['team_away_target_all_seasons'] / train['team_away_games_all_seasons']   # all seasons


параметры `'забитые\пропущенные голы за один\все сезон\ы'`

In [ ]:
# home_team забитые голы 
# за сезон 
train['team_home_scored_goals_one_season'] = train.groupby(['team_home', 'season'])['team_home_score'].transform('sum')
# за все сезоны
train['team_home_scored_goals_all_seasons'] = train.groupby('team_home')['team_home_score'].transform('sum')

# home_team пропущенные голы  
# за сезон 
train['team_home_missed_goals_one_season'] = train.groupby(['team_home', 'season'])['team_away_score'].transform('sum')
# за все сезоны
train['team_home_missed_goals_all_seasons'] = train.groupby('team_home',)['team_away_score'].transform('sum')
#data['team_home_score_out_season'] = data.groupby(['team_home', 'season'])['team_away_score'].transform('sum')

# home_away забитые голы 
# за сезон 
train['team_away_scored_goals_one_season'] = train.groupby(['team_away', 'season'])['team_away_score'].transform('sum')
# за все сезоны
train['team_away_scored_goals_all_seasons'] = train.groupby('team_away')['team_away_score'].transform('sum')

# home_away пропущенные голы  
# за сезон 
train['team_away_missed_goals_one_season'] = train.groupby(['team_away', 'season'])['team_home_score'].transform('sum')
# за все сезоны
train['team_away_missed_goals_all_seasons'] = train.groupby('team_away',)['team_home_score'].transform('sum')
#data['team_home_score_out_season'] = data.groupby(['team_home', 'season'])['team_away_score'].transform('sum')

Признаки 'Среднее забитых\пропущенных голов'

In [ ]:
# home_team
# за сезон
# забитые
train['f5-1'] = train['team_home_scored_goals_one_season'] / train['team_home_games_one_season']
# пропущенные
train['f5-2'] = train['team_home_missed_goals_one_season'] / train['team_home_games_one_season']
# за все сезоны
# забитые
train['f5-3'] = train['team_home_scored_goals_all_seasons'] / train['team_home_games_all_seasons']
# пропущенные
train['f5-4'] = train['team_home_missed_goals_all_seasons'] / train['team_home_games_all_seasons']

# away_team
# за сезон
# забитые
train['f6-1'] = train['team_away_scored_goals_one_season'] / train['team_away_games_one_season']
# пропущенные
train['f6-2'] = train['team_away_missed_goals_one_season'] / train['team_away_games_one_season']
# за все сезоны
# забитые
train['f6-3'] = train['team_away_scored_goals_all_seasons'] / train['team_away_games_all_seasons']
# пропущенные
train['f6-4'] = train['team_away_missed_goals_all_seasons'] / train['team_away_games_all_seasons']

Признаки `'Сухие игры'`

Доля матчей без забитых\пропущенных голов.

In [ ]:
#--------------------------------------------- team_home
#--------------------one_season               
# не забили
train['f712'] = ''
train['f711'] = train[['team_home', 'season', 'team_home_score']].loc[train['team_home_score'] == 0].groupby(['team_home', 'season']).transform('count') 
d = {
    (t.team_home, t.season): t.f711 for t in train.itertuples() if t.f711 in range(1, 100, 1)
}
train['f712'] = [d.get((t.team_home, t.season), t.f712) for t in train.itertuples()]
train['f712'] = train['f712'].replace('', 0) 
train['f7-1'] = train['f712'] / train['team_home_games_one_season']

# не пропустили
train['f722'] = ''
train['f721'] = train[['team_home', 'season', 'team_away_score']].loc[train['team_away_score'] == 0].groupby(['team_home', 'season']).transform('count') 
d = {
    (t.team_home, t.season): t.f721 for t in train.itertuples() if t.f721 in range(1, 100, 1)
}
train['f722'] = [d.get((t.team_home, t.season), t.f722) for t in train.itertuples()]
train['f722'] = train['f722'].replace('', 0)    # команда не сыграла в этом сезоне сухих матчей
train['f7-2'] = train['f722'] / train['team_home_games_one_season'] 

#--------------all seasons 
# не забили
train['f7-3'] = (train.team_home.map( train.groupby(train['team_home'][train['team_home_score'] == 0]).size())  / train['team_home_games_all_seasons']).fillna(0)   
# не пропустили
train['f7-4'] = (train.team_home.map( train.groupby(train['team_home'][train['team_away_score'] == 0]).size()) / train['team_home_games_all_seasons']).fillna(0)   


# -------------------------team_away
# ----------one_season               
# не забили
train['f752'] = ''
train['f751'] = train[['team_away', 'season', 'team_away_score']].loc[train['team_away_score'] == 0].groupby(['team_away', 'season']).transform('count') 
d = {
    (t.team_away, t.season): t.f751 for t in train.itertuples() if t.f751 in range(1, 100, 1)
}
train['f752'] = [d.get((t.team_away, t.season), t.f752) for t in train.itertuples()]
train['f752'] = train['f752'].replace('', 0)   
train['f7-5'] = train['f752'] / train['team_away_games_one_season'] 

# не пропустили
train['f762'] = ''
train['f761'] = train[['team_away', 'season', 'team_home_score']].loc[train['team_home_score'] == 0].groupby(['team_away', 'season']).transform('count') 
d = {
    (t.team_away, t.season): t.f761 for t in train.itertuples() if t.f761 in range(1, 100, 1)
}
train['f762'] = [d.get((t.team_away, t.season), t.f762) for t in train.itertuples()]
train['f762'] = train['f762'].replace('', 0)    # команда не сыграла в этом сезоне сухих матчей
train['f7-6'] = train['f762'] / train['team_away_games_one_season'] 


#-------------all seasons
# не забили
train['f7-7'] = (train.team_away.map( train.groupby(train['team_away'][train['team_home_score'] == 0]).size()) / train['team_away_games_all_seasons']).fillna(0)   
# не пропустили
train['f7-8'] = (train.team_away.map( train.groupby(train['team_away'][train['team_away_score'] == 0]).size()) / train['team_away_games_all_seasons']).fillna(0)   


Признак `'Средняя разница забитых\пропущенных'`

In [ ]:
#team_home
#one season
train['f8-1'] = train['f5-1'] - train['f5-2']
#all_seasons
train['f8-2'] = train['f5-3'] - train['f5-4']

#team_away
#one season
train['f8-3'] = train['f6-1'] - train['f6-2']
#all_seasons
train['f8-4'] = train['f6-3'] - train['f6-4']

Итого у нас получилось:

Параметры:
   - season 
   - количество побед\поражений\ничьих за сезон (team_home_target_season)

Признаки:
- f19 среднее очков в сезоне home
- f20 среднее очков с сезоне away
- f21 доля п\п\н home
- f22 доля п\п\н away
- f23           разница УБРАЛ
- f24           разница УБРАЛ
- f25 среднее забитые home
- f26 среднее пропущенные home
- f27 среднее забитые away
- f28 среднее пропущенные away
- f29 не забили home\n
- f30 не пропустили home\n
- F31 не забили away\n
- F32 не пропустили away\n

NaN'ов нет

In [ ]:
# 28
cols = ['f1-1', 'f1-2', 'f2-1', 'f2-2', 'f3-1', 'f3-2', 'f4-1', 
        'f4-2', 'f5-1', 'f5-2', 'f5-3', 'f5-4', 'f6-1', 'f6-2', 
        'f6-3', 'f6-4', 'f7-1', 'f7-2', 'f7-3', 'f7-4', 'f7-5', 
        'f7-6', 'f7-7', 'f7-8', 'f8-1', 'f8-2', 'f8-3', 'f8-4'] 

train[cols].info()       

нормализация

In [ ]:
for col in train[['f1-1', 'f1-2', 'f2-1', 'f2-2', 'f3-1', 'f3-2', 'f4-1', 'f4-2', 'f5-1', 'f5-2', 'f5-3', 'f5-4',
       'f6-1', 'f6-2', 'f6-3', 'f6-4', 'f7-1', 'f7-2', 'f7-3', 'f7-4', 'f7-5', 
       'f7-6', 'f7-7', 'f7-8', 'f8-1', 'f8-2', 'f8-3', 'f8-4']].columns.tolist():
    try:
        print(col, 'min: ', train[col].min(), 'max: ', train[col].max())
    except TypeError:
      print(col)

In [ ]:
#14
cols_for_norm = ['f1-1', 'f1-2', 'f2-1', 'f2-2', 'f5-1', 'f5-2', 'f5-3', 
                 'f5-4', 'f6-1', 'f6-2', 'f6-3', 'f6-4', 'f8-1', 'f8-2', 
                 'f8-3', 'f8-4']

scaler = MinMaxScaler()
train[cols_for_norm] = scaler.fit_transform(train[cols_for_norm]) 

In [ ]:
# set for one season
cols_one_season = ['f1-1', 'f2-1', 'f3-1', 'f4-1' ,'f5-1' ,'f5-2', 'f6-1',
                   'f6-2', 'f7-1', 'f7-2', 'f7-5', 'f7-6', 'f8-1', 'f8-3']

# set for all_seasons
cols_all_seasons = ['f1-2', 'f2-2', 'f3-2', 'f4-2', 'f5-3', 'f5-4', 'f6-3',
                    'f6-4', 'f7-3', 'f7-4', 'f7-7', 'f7-8', 'f8-2','f8-4']

#**Модель**

###Logistic Regression

Так как нам нужна вероятность каждого из трёх возможных исходов, то с самого начала я смотрел на LR. Она же и показала себя хорошо по сравнению с несколькими другими (Random Forrest, Xgboost) опробованными моделями.

Подобрать и потюнить алгоритм, особенно в данном соревновании, менее важно по сравнению с изучениeм и предобработкой данных. Поэтому этап выбора модели пропущен.

In [ ]:
lr = LogisticRegression()

param_grid = {
     'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['liblinear']}

# Create grid search object
gs = GridSearchCV(lr, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data
gs.fit(train[cols_one_season], train['target'])

In [ ]:
print(gs.best_score_)  # оценка на лучшем наборе параметров
print(gs.best_params_)  # лучший набор параметров 

0.7517014996900491
{'C': 11.288378916846883, 'penalty': 'l1', 'solver': 'liblinear'}


In [ ]:
# с лучшими из подобранных параметров заново создаётся модель
lr = LogisticRegression(solver='liblinear', penalty='l1', C = 11.3) 
preds = cross_val_predict(lr, train[cols_one_season], train['target'], n_jobs=-1, method="predict_proba")  
print("Score is {}".format(log_loss(train['target'], preds)))   

Score is 0.6122988668786341


In [ ]:
#'заново обучается'
lr.fit(train[cols_one_season], train['target']) 

LogisticRegression(C=11.3, penalty='l1', solver='liblinear')

#Признаки для **test**

Формируем `test` в двух окнах - `team_home` & `team_away`

Вот тут обнаружилось ещё несколько проблем, 
* Команда присутствует в `train`, но отсутствует в `test`, либо наоборот. 
* В `train` команда есть, но только `home`, а `test` только `away`. 
* Команда присутствует в `train` и `test`, но в разные сезоны.

Это очень сильно снижает качество предсказания! С другой стороны такое усложнение похоже на привет от организаторов, а значит я на правильном пути!

Вынужденно вводится важный параметр `marker`, который покажет степень присутствия команды в `train`. Объяснение в комментариях в следующей ячейке.

In [ ]:
test["team_home_marker"] = '' 

for i in range(0, test.shape[0], 1):
    if test['team_home'][i] in set(train['team_home']):
        if test['season'][i] in set(train[train['team_home'] == test['team_home'][i]]['season']):
            test["team_home_marker"][i] = 1        # 2043     сезоны совпали => признак будет на основании суммы по одному сезону
        else:
            test["team_home_marker"][i] = 2        # 4673     нет такого сезона => признак будут на основании суммы всех сезонов
    else:
        test["team_home_marker"][i] = 3            # 495=7%   такой команды из test нет в train

In [ ]:
test["team_away_marker"] = '' 

for i in range(0, test.shape[0], 1):
    if test['team_away'][i] in set(train['team_away']):
        if test['season'][i] in set(train[train['team_away'] == test['team_away'][i]]['season']):
            test["team_away_marker"][i] = 4                  # 2043         мэпинг по сезону
        else:
            test["team_away_marker"][i] = 5                  # 4677         мэпинг среднего по всем сезонам
    else:
        test["team_away_marker"][i] = 6                      # 491    

`team_home`

In [ ]:
for c in test[cols_one_season]:
    del test[c]

In [ ]:
test[['f1-1', 'f3-1', 'f5-1', 'f5-2', 'f7-1', 'f7-2', 'f8-1']].head()

In [ ]:
# имена колонок должны совпадать с train[cols_one_season]
test['f1-1'], test['f3-1'], test['f5-1'], test['f5-2'], test['f7-1'], test['f7-2'], test['f8-1'] = '','','','','','',''


# team_home
for i in range(0, test.shape[0], 1):
    #if test['team_home'][i] in set(train['team_home']):
        #if test['season'][i] in set(train[train['team_home'] == test['team_home'][i]]['season']):
        if test["team_home_marker"][i] == 1:
            #test["team_home_marker"][i] = 1        # 2043     сезоны совпали => признак будет на основании суммы по одному сезону
            test['f1-1'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f1-1'].tolist()[0]
            test['f3-1'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f3-1'].tolist()[0]
            test['f5-1'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f5-1'].tolist()[0]
            test['f5-2'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f5-2'].tolist()[0]
            test['f7-1'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f7-1'].tolist()[0]
            test['f7-2'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f7-2'].tolist()[0]
            test['f8-1'][i] = train.loc[(train['team_home'] == test['team_home'][i]) & (train['season'] == test['season'][i])]['f8-1'].tolist()[0]
        elif test["team_home_marker"][i] == 2:
            #test["team_home_marker"][i] = 2        # 4673     нет такого сезона => признак будут на основании суммы всех сезонов
            test['f1-1'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f1-2'].tolist()[0]
            test['f3-1'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f3-2'].tolist()[0]
            test['f5-1'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f5-3'].tolist()[0]
            test['f5-2'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f5-4'].tolist()[0]
            test['f7-1'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f7-3'].tolist()[0]
            test['f7-2'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f7-4'].tolist()[0]
            test['f8-1'][i] = train.loc[train['team_home'] == test['team_home'][i]]['f8-2'].tolist()[0]
        elif test["team_home_marker"][i] == 3:           #495 (~7%)   такой команды из test нет в train
            # потом доделать среднее по чемпу и сезону
            test['f1-1'][i] = train['f1-1'].mean()
            test['f3-1'][i] = train['f3-1'].mean()
            test['f5-1'][i] = train['f5-1'].mean()
            test['f5-2'][i] = train['f5-2'].mean()
            test['f7-1'][i] = train['f7-1'].mean()
            test['f7-2'][i] = train['f7-2'].mean()
            test['f8-1'][i] = train['f8-1'].mean()


`team_away`

In [ ]:
# имена колонок должны совпадать с train[cols_one_season]
test['f2-1'], test['f4-1'], test['f6-1'], test['f6-2'], test['f7-5'], test['f7-6'], test['f8-3'] = '','','','','','',''


# team_away
for i in range(0, test.shape[0], 1):
        if test["team_away_marker"][i] == 4:
            test['f2-1'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f2-1'].tolist()[0]
            test['f4-1'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f4-1'].tolist()[0]
            test['f6-1'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f6-1'].tolist()[0]
            test['f6-2'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f6-2'].tolist()[0] 
            test['f7-5'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f7-5'].tolist()[0]
            test['f7-6'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f7-6'].tolist()[0]
            test['f8-3'][i] = train.loc[(train['team_away'] == test['team_away'][i]) & (train['season'] == test['season'][i])]['f8-3'].tolist()[0]
        elif test["team_away_marker"][i] == 5:
            test['f2-1'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f2-2'].tolist()[0] 
            test['f4-1'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f4-2'].tolist()[0]
            test['f6-1'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f6-3'].tolist()[0]
            test['f6-2'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f6-4'].tolist()[0]
            test['f7-5'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f7-7'].tolist()[0] 
            test['f7-6'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f7-8'].tolist()[0] 
            test['f8-3'][i] = train.loc[train['team_away'] == test['team_away'][i]]['f8-4'].tolist()[0]
        elif test["team_away_marker"][i] == 6:           #495 (~7%)   такой команды из test нет в train
            # потом доделать среднее по чемпу и сезону
            test['f2-1'][i] = train['f2-1'].mean()
            test['f4-1'][i] = train['f4-1'].mean() 
            test['f6-1'][i] = train['f6-1'].mean()
            test['f6-2'][i] = train['f6-2'].mean()
            test['f7-5'][i] = train['f7-5'].mean()
            test['f7-6'][i] = train['f7-6'].mean()
            test['f8-3'][i] = train['f8-3'].mean()


In [ ]:
test_cols = ['f1-1', 'f2-1', 'f3-1', 'f4-1' ,'f5-1' ,'f5-2', 'f6-1',
                   'f6-2', 'f7-1', 'f7-2', 'f7-5', 'f7-6', 'f8-1', 'f8-3']

In [ ]:
test[test_cols].head()

In [ ]:
test[['tour', 'team_home', 'team_away', 'time', 'champ','season', 'team_home_marker', 'team_away_marker']].head()

### submit

Финал, делаем сабмит.

In [ ]:
submit = pd.DataFrame(lr.predict_proba(test[test_cols]), columns=["draw", "win", "lose"]).round(3)
submit.to_csv("submit.csv")

In [ ]:
#C:\Users\Mirsky\Desktop\Compts\boosters_football
# df.to_csv('raw_data.csv', index=False)

submit.to_csv(r"C:\submit.csv",index=False)

In [ ]:
submit.head() 